<a href="https://colab.research.google.com/github/nuot/AL-Clean/blob/main/AssistantListing_ObliClean_InProgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assistant Listing Obligation data cleaning project Version1** 
<br>

Author: Nuo Tian



**Last Updated: 03/16/2021**

Ask user input for the file name, target the obligation column

In [81]:
import pandas as pd
import re

filename = input("Enter name of input file (including the file extension):")
data = pd.read_csv(filename)

Enter name of input file (including the file extension):AssistanceListings_DataGov_PUBLIC_WEEKLY_20210213 - AssistanceListings_DataGov_PUBLIC_WEEKLY_20210213.csv


Loop through column names, find the column name contains 'Obligation' and get the column name 

In [82]:
colnames = list(data.columns)
obligations_colname = [col for col in colnames if re.findall(r'Obligation', col)][0]
obligations = data[obligations_colname]

find all types in each row

In [83]:
types = []
for row in obligations:
  a = re.findall(r'\((.*?)\) FY',str(row))
  if a:
    types.append(a)
  else:
    types.append('None')

In [84]:
types_dedup = []
for type1 in types:
    type1 = list(dict.fromkeys(type1))
    types_dedup.append(type1)

In [85]:
data['Obligation Types'] = types_dedup

Next working on seprating the $amount for each obligation type

In [86]:
def listToStringWithoutBrackets(list1):
    return str(list1).replace('[','').replace(']','').replace("'","")

In [87]:
Obli_types = []
for i in range(len(data['Obligation Types'])):
  res = listToStringWithoutBrackets(data['Obligation Types'][i])
  Obli_types.append(res)

In [88]:
data['Obligation Types'] = Obli_types

In [89]:
maxlen = max(list(map(lambda x: len(x.split(",")) ,data['Obligation Types'])))
cols = ["Obligation"+str(x)   for x in range(maxlen)]
datalist = list(map(lambda x: x.split(","), data['Obligation Types']))
newdf = pd.DataFrame(data=datalist, columns=cols) 
data = pd.concat([data, newdf], axis=1) 

In [ ]:
#mask = data.applymap(lambda x: x is None)
#cols = data.columns[(mask).any()]
#for col in data[cols]:
    #data.loc[mask[col], col] = ''

Amount

In [90]:
obli1 = data['Obligations (122)'].str.split(')', expand=True)

In [91]:
df = pd.DataFrame()

i = 0
for i in range(len(obli1.columns)):
  val = []
  for row in obli1[i]:
    a = re.findall(r'FY [0-9][0-9]\$[^\]]+[.%]?',str(row))
    b = re.findall(r'FY [0-9][0-9] est \$[^\]]+[.%]?',str(row))
    if a:
      val.append(a)
    elif b:
      val.append(b)
    else:
      val.append('')
  df.insert(i, i, val, True) 


In [92]:
df = df.applymap(str)

In [94]:
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace("['", "")
        text = text.replace("- ']", "")
    return text

In [95]:
i = 0
while i < len(df.columns):
  df[i] = df[i].apply(remove_punctuations)
  i +=1

In [98]:
for j in range(maxlen):
  FY19 = []
  FY20 = []
  FY20_Flag = []
  FY21 = []
  FY21_Flag = []
  name = 'Obligation'+str(j)
  for i in range(len(df[j])):  
    a = re.findall(r'FY 19\$[^\]]+[.%]?',df[j+1][i])
    b = re.findall(r'FY 20\$[^\]]+[.%]?',df[j+1][i])
    c = re.findall(r'FY 20 est \$[^\]]+[.%]?', df[j+1][i])
    d = re.findall(r'FY 21\$[^\]]+[.%]?',df[j+1][i])
    e = re.findall(r'FY 21 est \$[^\]]+[.%]?', df[j+1][i])
    if a :
      FY19.append(a[0].split(';')[0].replace('FY 19$',''))
    else:
      FY19.append('')
    if b:
      FY20.append(b[0].split(';')[0].replace('FY 20$',''))
      FY20_Flag.append(0)
    elif c:
      FY20.append(c[0].split(';')[0].replace('FY 20 est $',''))
      FY20_Flag.append(1)
    else:
      FY20.append('')
      FY20_Flag.append('')
    
    if d:
      FY21.append(d[0].split(';')[0].replace('FY 21$',''))
      FY21_Flag.append(0)
    elif e:
      FY21.append(e[0].split(';')[0].replace('FY 21 est $',''))
      FY21_Flag.append(1)
    else:
      FY21.append('')
      FY21_Flag.append('')

  data.insert(data.columns.get_loc(name) + 1, 'FY21_Flag'+ str(j), FY21_Flag)
  data.insert(data.columns.get_loc(name) +1, 'FY21_'+ str(j), FY21)
  data.insert(data.columns.get_loc(name) +1, 'FY20_Flag'+ str(j), FY20_Flag)
  data.insert(data.columns.get_loc(name) +1, 'FY20_'+ str(j), FY20)
  data.insert(data.columns.get_loc(name) +1, 'FY19_'+ str(j), FY19)

In [100]:
filename

'AssistanceListings_DataGov_PUBLIC_WEEKLY_20210213 - AssistanceListings_DataGov_PUBLIC_WEEKLY_20210213.csv'

In [107]:
reference_Data = re.findall('\d+', filename)[0]

In [108]:
data['Reference_Date'] = reference_Data

In [1155]:
data.to_csv('AssistanceListingFinal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>